# Secret Sharing

Secret sharing is a technique widely used in secure multiparty computation. It relies on sharing the responsability of a certain information by simply splitting it in several shares. As always we explain by example.

1. In the first part we will work on additive secret sharing
2. A more complex technique, polynomial reconstruction and Shamir secret sharing.


## Additive Secret Sharing

As said previously, secret sharing consists (at least at this stage) in splitting the information into shares. First let's choose a random integer all the parties agree on:

In [26]:
from crypt import RandomPrime
from random import seed, randrange

seed(1)

bits = 8
p = RandomPrime(bits, 40)

# Although in this example we work with a random number it is not strictly necessary
print(f"We've chosen a random prime p={p}")

We've chosen a random prime p=193


We choose the number of parties $N$ that will participate in the calculation and the secret we want to split. Then we generate N-1 random numbers in between 0 and $p$ and finally substract the secret to generate the Nth share, this way when we sum the shares they will add up to the secret.

In [32]:
N = 5
secret = 100

def AdditiveShares(secret, N, p):
    # generate N-1 random shares
    shares = [randrange(p) for _ in range(N-1)]
    shares.append((secret-sum(shares))%p)
    return shares

def ReconstructAdditiveShares(shares, p):
    return (sum(shares)%p)

shares = AdditiveShares(secret, N, p)
reconstruction = ReconstructAdditiveShares(shares, p)
print(f"Splitting to shares:\n\nsecret={secret}\nsplits={shares}\nsum_shares={reconstruction}")

Splitting to shares:

secret=100
splits=[68, 184, 58, 151, 25]
sum_shares=100


What we've done here is splitting the secret into N users so that when they all share their shares they can reconstruct the secret, no $N-1$ parties can reconstruct!. Beautiful right?

Now, how much information can $N-1$ parties gain from the original secret? Here's the best, there's no information. This is true due to the randomicity of this process, since $N-1$ are chosen completely at random from a uniform distribution they contain no information and by construction the Nth number also. This concept is exactly the same as the one time pad encoding we have seen previously.

We can define addition and substraction on shares. 

In [55]:
def AdditiveSharesAdd(x, y, p):
    return [(i+j)%p for i, j in zip(x,y)]

def AdditiveSharesSubstract(x, y, p):
    return [(i-j)%p for i, j in zip(x,y)]

It is possible to outsource a summation or a substraction by sharing the values of two integers in the following way

In [52]:
x = randrange(0, p)
y = randrange(0, p)

print(f"I want to add secretly x and y:\nx={x}\ny={y}\n")
print(f"First task is to generate random shares for x and y and send them to the {N} outsourced parties:")
x_shares = AdditiveShares(x, N, p)
y_shares = AdditiveShares(y, N, p)

x_rec = ReconstructAdditiveShares(x_shares, p)
y_rec = ReconstructAdditiveShares(y_shares, p)

print(f"x_shares = {x_shares} which reconstructed is {x_rec}")
print(f"y_shares = {y_shares} which reconstructed is {y_rec}\n")

I want to add secretly x and y:
x=98
y=87

First task is to generate random shares for x and y and send them to the 5 outsourced parties:
x_shares = [107, 48, 66, 27, 43] which reconstructed is 98
y_shares = [64, 186, 130, 53, 40] which reconstructed is 87



Now Alice has to send each share of X and Y to a different party

In [53]:
for i, (xs, ys) in enumerate(zip(x_shares, y_shares)):
    print(f"Party {i} gets the values x_{i}={xs} and y_{i}={ys}")

Party 0 gets the values x_0=107 and y_0=64
Party 1 gets the values x_1=48 and y_1=186
Party 2 gets the values x_2=66 and y_2=130
Party 3 gets the values x_3=27 and y_3=53
Party 4 gets the values x_4=43 and y_4=40


Each party calculates their sum of their shares and send it back to Alice

In [59]:
x_y_sum_shares = AdditiveSharesAdd(x_shares, y_shares, p)
print(f"The final sum of shares is: \nx_y_sum={x_y_sum_shares}")

The final sum of shares is: 
x_y_sum=[171, 41, 3, 80, 83]


Now is time to reconstruct the shares and find the secret

In [60]:
x_y_sum = ReconstructAdditiveShares(x_y_sum_shares, p)

assert(x+y==x_y_sum)

print(f"The reconstruction of the secret is {x_y_sum}, the sum of x={x} and y={y}")

The reconstruction of the secret is 185, the sum of x=98 and y=87
